In [ ]:
!pip install transformers googletrans==4.0.0-rc1

import json
import os
import numpy as np

import torch
from torchvision import models, transforms
from transformers import AutoTokenizer, AutoModel, BertTokenizer, BertModel

from PIL import Image
from googletrans import Translator
from sklearn.metrics.pairwise import cosine_similarity
import urllib.request

import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer


from tqdm import tqdm
import time

In [ ]:
img_path = "images.zip"
cap_path = "captions.json"

'''if not os.path.exists(img_path):
    img_url = "https://data.mendeley.com/public-files/datasets/rxxch9vw59/files/220ffc5a-8645-4c0b-bd1b-97188836ed3b/file_downloaded"
    img_path, _ = urllib.request.urlretrieve(img_url, img_path)
    print(f"Downloaded {img_path}")
else:
    print(f"File {img_path} already exists.")'''

if not os.path.exists(cap_path):
    cap_url = "https://data.mendeley.com/public-files/datasets/rxxch9vw59/files/9b3e789a-5a5c-48b3-8a2c-2c91e9307c2a/file_downloaded"
    req = urllib.request.Request(cap_url, headers={"User-Agent": "Mozilla/5.0"})
    with urllib.request.urlopen(req) as response, open(cap_path, 'wb') as out_file:
        out_file.write(response.read())
    print(f"Downloaded {cap_path}")
else:
    print(f"File {cap_path} already exists.")

Downloaded captions.json


In [ ]:
if os.path.exists(cap_path):
    with open(cap_path, 'r') as f:
        data = json.load(f)
    print("Preview of captions.json content:")
    print(json.dumps(data, indent=2)[:500])  # Print the first 500 characters

Preview of captions.json content:
[
  {
    "filename": "1.png",
    "caption": [
      "\u09a4\u09bf\u09a8 \u099c\u09a8 \u09ae\u09c7\u09df\u09c7 \u09ae\u09be\u09a8\u09c1\u09b7 \u0986\u099b\u09c7\u0964 \u098f\u0995 \u099c\u09a8 \u09a6\u09be\u09dc\u09bf\u09df\u09c7 \u0986\u099b\u09c7 \u0986\u09b0 \u09a6\u09c1\u0987 \u099c\u09a8 \u09ac\u09b8\u09c7 \u0986\u099b\u09c7\u0964",
      "\u098f\u0995\u099f\u09bf \u09b9\u09b2\u09c1\u09a6 \u099c\u09be\u09ae\u09be \u09aa\u09be\u09df\u099c\u09be\u09ae\u09be \u09aa\u09b0\u09be \u09ae\u09b9\u0


In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
def batch_translate(captions, delimiter="|||"):
    """Translate captions in a batch using a delimiter."""
    translator = Translator()

    if not captions:
        raise ValueError("Caption batch is empty.")

    valid_captions = [caption for caption in captions if caption]  # Filter out None or empty captions

    if not valid_captions:
        raise ValueError("All captions in the batch are empty or None.")

    try:
        concatenated = f" {delimiter} ".join(captions)
        translated = translator.translate(concatenated, src='bn', dest='en')

        if not translated or not translated.text:
            raise ValueError("Translation returned no results.")
        time.sleep(2)
        return translated.text.split(delimiter)
    except Exception as e:
        print(f"Error during translation: {e}")
        return [""] * len(captions)

In [ ]:
def preprocess_text(text):
    """Clean text, remove stopwords, and lemmatize."""
    REPLACE_BY_SPACE_RE = re.compile(r'[/(){}\[\]\|@,;]')
    SYMBOLS_RE = re.compile(r'[^0-9a-z #+_]')
    NUMBERS = re.compile(r'\d+')
    STOPWORDS = set(stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()

    # Lowercase the text
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(' ', text)
    text = SYMBOLS_RE.sub('', text)
    text = NUMBERS.sub('', text)

    # Remove stopwords and lemmatize
    tokens = [word for word in text.split() if word not in STOPWORDS]
    tokens = [lemmatizer.lemmatize(token) for token in tokens]

    return tokens

In [ ]:
if os.path.exists(cap_path):
    with open(cap_path, 'r') as f:
        data = json.load(f)

    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    model = BertModel.from_pretrained("bert-base-uncased")
    model.eval()
    embeddings = {}

    batch_size = 100
    all_captions = [entry['caption'][0] for entry in data]
    filenames = [entry['filename'] for entry in data]

    for i in tqdm(range(0, len(all_captions), batch_size), desc="Processing batches"):
        batch_captions = all_captions[i:i + batch_size]
        batch_filenames = filenames[i:i + batch_size]

        # Translate captions in the current batch
        translated_captions = batch_translate(batch_captions)

        print(f"Batch {i} translations:")
        for original, translated in zip(batch_captions[:5], translated_captions[:5]):
            print(f"Original: {original}")
            print(f"Translated: {translated}")
            print("------")

        for filename, english_caption in zip(batch_filenames, translated_captions):
            cleaned_tokens = preprocess_text(english_caption)
            tokenized_input = tokenizer(" ".join(cleaned_tokens), return_tensors="pt", padding=True, truncation=True)

            with torch.no_grad():
                outputs = model(**tokenized_input)
                embedding = outputs.last_hidden_state.mean(dim=1).squeeze()  # Take mean of token embeddings

            embeddings[filename] = embedding

    # Save embeddings for later use
    np.save("caption_embeddings.npy", embeddings)
    print("Saved caption embeddings to 'caption_embeddings.npy'")

Processing batches:   0%|          | 0/92 [00:00<?, ?it/s]

Batch 0 translations:
Original: তিন জন মেয়ে মানুষ আছে। এক জন দাড়িয়ে আছে আর দুই জন বসে আছে।
Translated: There are three girls people.One is standing and two are sitting.
------
Original: অনেক মেয়ে মানুষ বসে আছে।
Translated: Many girls are sitting.
------
Original: অনেক মানুষ একসাথে বসে কাজ করছে।
Translated: Many people are working together.
------
Original: ছয় জন মানুষ দাড়িয়ে আছে।
Translated: Six people are standing.
------
Original: এক জন মেয়ে মানুষ মাথায় ঘোমটা দিয়ে কাজ করছে। মাটিতে বিভিন্ন রঙের মসলা আছে।
Translated: One girl is working with a sweat on the head.There are spices of different colors on the ground.
------


Processing batches:   1%|          | 1/92 [00:11<17:40, 11.65s/it]

Batch 100 translations:
Original: একজন মহিলা মানুষ কাজ করছে।
Translated: A female man is working.
------
Original: আলো ও আধারের মাঝে দুইজন মানুষ দেখা যাচ্ছে।
Translated: Two people are seen in light and light.
------
Original: একজন বালিকা হাসছে।
Translated: A girl is smiling.
------
Original: একজন পুরুষ মানুষ কাজ করছে।
Translated: A male man is working.
------
Original: নয় জন পুরুষ মানুষ দাঁড়িয়ে আছে।
Translated: Nine men are standing.
------


Processing batches:   2%|▏         | 2/92 [00:22<16:55, 11.28s/it]

...

Error during translation: the JSON object must be str, bytes or bytearray, not NoneType
Batch 9000 translations:
Original: একটা বাচ্চা ছেলে দাঁড়িয়ে দাঁত বের করে হাসছে।
Translated: 
------
Original: নদীতে কয়েকটা নৌকা চলতে দেখা যাচ্ছে।
Translated: 
------
Original: কালো স্কার্ফ পরা একটা ছোট মেয়ে হাতে একটা সিলেট নিয়ে হাঁসি মুখে দাঁড়িয়ে আছে।
Translated: 
------
Original: হলুদ ফুলে ভরা সরিষা ক্ষেতের মাঝের রাস্তা দিয়ে দুইজন ছেলে একটা সাইকেলে বসে যাচ্ছে আর পিছন থেকে একটা ছেলে সেই সাইকেল ধাক্কা দিচ্ছে।
Translated: 
------
Original: নদীর পাড়ে দাঁড়িয়ে একজন লোক পানিতে কাপড় ধুচ্ছে।
Translated: 
------


Processing batches:  99%|█████████▉| 91/92 [16:41<00:08,  8.87s/it]

Batch 9100 translations:
Original: দুইপাশে গাছপালার সারি আর মাঝের রাস্তা দিয়ে একজন মেয়ে হেঁটে যাচ্ছে।
Translated: A girl is walking along the rows of vegetation and middle road on both sides.
------
Original: একটা রাস্তা দিয়ে পাঁচজন লোক হেঁটে যাচ্ছে।
Translated: Five people are walking along one road.
------
Original: একটা বাচ্চা ছেলে একা দাঁড়িয়ে আছে।
Translated: A baby boy is standing alone.
------
Original: বৃষ্টিতে একটা মেয়ে ছাতা মাথায় ধরে হাত বৃষ্টিতে ভিজাচ্ছে।
Translated: In the rain, a girl's umbrella holds her hands wet in the rain.
------
Original: একটা ছোট মেয়ে হাতে একটা খাতা নিয়ে হাঁসি দিয়ে তাকিয়ে আছে।
Translated: A little girl looks at a duck with a book in her hand.
------


Processing batches: 100%|██████████| 92/92 [16:51<00:00, 11.00s/it]

Saved caption embeddings to 'caption_embeddings.npy'


In [ ]:
import numpy as np

# Load the embeddings from the .npy file
embeddings = np.load("caption_embeddings.npy", allow_pickle=True).item()

# Check the type and keys
print(f"Type of embeddings: {type(embeddings)}")
print(f"Number of entries: {len(embeddings)}")

# Print the first few keys and their corresponding embeddings
for i, (filename, embedding) in enumerate(embeddings.items()):
    print(f"Filename: {filename}")
    print(f"Embedding shape: {embedding.shape}")
    print(f"Embedding (first 5 values): {embedding[:5]}")
    print()  # Add a blank line for better readability
    if i >= 4:  # Limit to the first 5 items
        break

Type of embeddings: <class 'dict'>
Number of entries: 9154
Filename: 1.png
Embedding shape: (768,)
Embedding (first 5 values): [-0.00922781 -0.22185887 -0.34482703  0.01650961 -0.29198343]

Filename: 2.png
Embedding shape: (768,)
Embedding (first 5 values): [-0.00922781 -0.22185887 -0.34482703  0.01650961 -0.29198343]

Filename: 3.png
Embedding shape: (768,)
Embedding (first 5 values): [-0.00922781 -0.22185887 -0.34482703  0.01650961 -0.29198343]

Filename: 4.png
Embedding shape: (768,)
Embedding (first 5 values): [-0.00922781 -0.22185887 -0.34482703  0.01650961 -0.29198343]

Filename: 5.png
Embedding shape: (768,)
Embedding (first 5 values): [-0.00922781 -0.22185887 -0.34482703  0.01650961 -0.29198343]

